## 1.生成数据集

In [1]:
from mxnet import nd, autograd

# 生成数据集
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

## 2.读取数据

In [2]:
from mxnet.gluon import data as gdata

batch_size = 10
# 将训练数据的特征和标签组合
dataset = gdata.ArrayDataset(features, labels)
# 随机读取小批量
data_iter = gdata.DataLoader(dataset, batch_size, shuffle=True)

In [3]:
for X, y in data_iter:
    print(X, y)
    break


[[ 0.44712448  0.28236017]
 [ 0.46851125  0.81799334]
 [-0.5617653   0.49843073]
 [ 0.6902953   0.15144643]
 [-0.8112422   0.1232819 ]
 [ 0.985972    0.98455536]
 [ 1.7655449  -0.7190822 ]
 [-0.00294514 -0.8033466 ]
 [-0.9203057  -0.25793338]
 [-2.7653847  -0.7842745 ]]
<NDArray 10x2 @cpu(0)> 
[ 4.110798   2.3493972  1.3927041  5.073271   2.1691186  2.8146505
 10.1846     6.934895   3.2231455  1.3224878]
<NDArray 10 @cpu(0)>


## 3.定义模型

In [4]:
from mxnet.gluon import nn

net = nn.Sequential()

In [5]:
# 向名为net的Sequential实例中添加层，这是一个全连接层，输出个数为1，无需指定输入的形状，模型会自动推断
net.add(nn.Dense(1))

## 4.初始化模型参数

In [6]:
from mxnet import init

# 权重参数随机采样于均值为0，标准差为0.01的正态分布，偏差参数则全部初始化为0
net.initialize(init.Normal(sigma = 0.01))

## 5.定义损失函数

In [8]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss()   # 平方损失又称L2范数损失

## 6.定义优化算法

In [9]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## 7.训练模型

In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch + 1, l.mean().asnumpy()))


epoch 1, loss: 0.035018
epoch 2, loss: 0.000127
epoch 3, loss: 0.000049


In [11]:
dense = net[0]
true_w, dense.weight.data(), true_b, dense.bias.data()

([2, -3.4],
 
 [[ 1.9994506 -3.3996866]]
 <NDArray 1x2 @cpu(0)>,
 4.2,
 
 [4.1994624]
 <NDArray 1 @cpu(0)>)